# Select database (i.e. staging, production, local authenticated, local unauthenticated)

In [ ]:
import requests
from obi_auth import get_token
from entitysdk import Client, ProjectContext, models
from enum import Enum

class DatabaseEnvironment(Enum):
    STAGING_DATABASE = "staging"
    # PRODUCTION_DATABASE = "production"
    # LOCAL_UNAUTHENTICATED = "local_unauthenticated"
    # Note: LOCAL_AUTHENTICATED is not supported by entitycore because the project/vlab ids do not exist in your keycloak groups

obi_one_api_url = "http://127.0.0.1:8100"

add_morphology_asset = False

database_environment = DatabaseEnvironment.STAGING_DATABASE
# database_environment = DatabaseEnvironment.PRODUCTION_DATABASE
# database_environment = DatabaseEnvironment.LOCAL_UNAUTHENTICATED

if database_environment == DatabaseEnvironment.STAGING_DATABASE:
    """
    - Uncomment in obi-one/.env.run-local before "make run-local": 
        export ENTITYCORE_URL=https://staging.openbraininstitute.org/api/entitycore
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"
    virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    project_id="2720f785-a3a2-4472-969d-19a53891c817"

# elif database_environment == DatabaseEnvironment.PRODUCTION_DATABASE:
#     """
#     For future: EntityCore not currently in production.
#     - Uncomment in obi-one/.env.run-local before "make run-local": 
#         export ENTITYCORE_URL=https://www.openbraininstitute.org/api/entitycore
#     """
#     entitycore_api_url = "https://www.openbraininstitute.org/api/entitycore"

elif DatabaseEnvironment.LOCAL_UNAUTHENTICATED:
    """
    Not yet tested.
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk)
    """
    entitycore_api_url = "http://127.0.0.1:8000"
    add_morphology_asset = True
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730"
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6"
    # virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    # project_id="2720f785-a3a2-4472-969d-19a53891c817"
    
else:
    raise ValueError(f"Unsupported environment: {database_environment}")


token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

# Find a morphology

In [13]:
from entitysdk.models.morphology import (
    ReconstructionMorphology,
)
kwargs = {}
reconstruction_morphologies = client.search_entity(
    entity_type=ReconstructionMorphology, query=kwargs, limit=10
).all()

reconstruction_morphology_id = reconstruction_morphologies[4].id
print(reconstruction_morphology_id)

02604cdb-e008-45c4-87a1-d61d5df88b06


# Fetch a morphology

In [8]:
from entitysdk.models.morphology import (
    ReconstructionMorphology,
)
kwargs = {}
reconstruction_morphology = client.get_entity(
    entity_id=reconstruction_morphology_id, entity_type=ReconstructionMorphology
    # entity_type=ReconstructionMorphology, query=kwargs, limit=10
)

# reconstruction_morphology_id = reconstruction_morphologies[1].id
print(reconstruction_morphology)

assets=[Asset(id=UUID('4eea423a-16a5-4412-8e53-89c288458675'), update_date=None, creation_date=None, path='737158657.asc', full_path='public/a98b7abc-fc46-4700-9e3d-37137812c730/0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6/assets/reconstruction_morphology/02604cdb-e008-45c4-87a1-d61d5df88b06/737158657.asc', is_directory=False, content_type='application/asc', size=1040738, sha256_digest='5dbd34b8d0816d3ab345b63791a91a104e7c66a43cb0534ec4dc7c5e973b8c77', status='created', meta={}), Asset(id=UUID('0cc8cb57-774b-4fb5-b95e-d0b99e317d48'), update_date=None, creation_date=None, path='737158657.swc', full_path='public/a98b7abc-fc46-4700-9e3d-37137812c730/0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6/assets/reconstruction_morphology/02604cdb-e008-45c4-87a1-d61d5df88b06/737158657.swc', is_directory=False, content_type='application/swc', size=1285804, sha256_digest='aadc2f4679696da1612afd94ee96c14b268a67b44a0b7d2acbea0e954393a188', status='created', meta={}), Asset(id=UUID('a1762c14-130a-4498-b44d-8a60a0447f31'),

In [ ]:
reconstruction_morphology_id = reconstruction_morphology.id
print(reconstruction_morphology_id)

In [ ]:
import tempfile
from pathlib import Path
import io

if add_morphology_asset:
    with tempfile.TemporaryDirectory() as tdir:

        src = Path("single_cell_simulations/components/morphologies/C060114A5.swc")
        file2 = Path(tdir, "morphology.swc")
        # print(src.read_text())
        file2.write_text(src.read_text())

        # use an in-memory buffer to upload second asset
        buffer = io.BytesIO(b"morph bytes buffer")

        asset2 = client.upload_content(
            entity_id=reconstruction_morphology_id,
            entity_type=ReconstructionMorphology,
            file_content=buffer,
            file_name="morphology.swc",
            file_content_type="application/swc",
        )

        print(asset2)


# Call the declared morphology metrics endpoint

In [9]:
# Construct the full endpoint URL
url = f"{obi_one_api_url}/declared/neuron-morphology-metrics/{reconstruction_morphology_id}"

# Set headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Make the GET request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")


Success: {'aspect_ratio': 0.46938310712331804, 'circularity': 0.6764430614289079, 'length_fraction_above_soma': 0.5866567492485046, 'max_radial_distance': 984.980224609375, 'number_of_neurites': 8, 'soma_radius': 5.548399925231934, 'soma_surface_area': 386.85247802734375}


# Call the generated morphology metrics endpoint

In [ ]:
import requests

# Construct the full endpoint URL
url = f"{obi_one_api_url}/generated/morphology-metrics-run-grid"

# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Construct request body — adjust this with the actual morphology metrics form data!
# Placeholder example:
request_body = {
  "type": "MorphologyMetricsForm",
  "initialize": {
    "type": "MorphologyMetricsForm.Initialize",
    "morphology": {
      "id_str": str(reconstruction_morphology_id),
    }
  }
}

# Make the POST request
response = requests.post(url, headers=headers, json=request_body)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")